In [14]:
#Je reviens à quelque chose de beaucoup plus simple. On va prouver que l'on obtient le bon FF pour STF classique. 
#Travaillons en mode dense, facile.
# This first version is for the classical STF. 
# Now, try to get to the domain-based STF...

import bempp.api 
import numpy as np
import mtf

from bempp.api.operators.boundary.sparse import identity
from bempp.api.assembly.blocked_operator import BlockedOperator, GeneralizedBlockedOperator

from mtf.utils import bhmie
from mtf. config import config
from mtf.functions import define_bempp_functions
from mtf.assembly.operators import GeneralizedBlockedOperator

from matplotlib import pyplot as plt
from decimal import Decimal
from bempp.api import ZeroBoundaryOperator

bempp.api.enable_console_logging()

from mtf.utils import bhmie
from mtf.config import config
mtf.config.set_case("B")

k0, k1 = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_r = config["eps_rel"]
mu_r = config["mu_rel"]
mu0 = config["mu_ext"]

mu1 = mu_r * mu0

eta_r = np.sqrt(mu_r / eps_r)



sphere0 = mtf.shapes.shapes.halves(domain=1, h=0.2)
sphere1 =  mtf.shapes.shapes.halves(domain=2, h=0.2)

grid = bempp.api.grid.union([sphere0, sphere1], domain_indices = [10,20])
#grid = bempp.api.shapes.multitrace_sphere(h=1)

M = 3

#segments = [ [0] , [1]]
segments = [ [10] , [20]]
swapped_normals = [[], []]

frequency = 300E6 # 300Mhz

vacuum_permittivity = 8.854187817E-12
vacuum_permeability = 4 * np.pi * 1E-7

k_ext = k0
k_int = k1

tangential_trace, neumann_trace = define_bempp_functions(config)

from bempp.api.operators.boundary.maxwell import multitrace_operator
from bempp.api.operators.boundary.sparse import multitrace_identity


bempp:HOST:INFO: Created grid with id d6f30eef-3abe-4d78-a5ee-22d55847ad06. Elements: 636. Edges: 954. Vertices: 320
bempp:HOST:INFO: Created grid with id 5db2af11-76cd-4db8-8707-bfa6f9eb3d84. Elements: 640. Edges: 960. Vertices: 322
bempp:HOST:INFO: Created grid with id 44068003-e6dd-455a-acad-c619ef7ca5a2. Elements: 1276. Edges: 1914. Vertices: 642


In [15]:
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
eta = np.sqrt(mu_r / eps_r)


multitrace_ops = []
multitrace_ops0 = []
# > Assemble all diagonal operators
for index in range(M-1):
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[index], rA[index], tA[index], k_int)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[index], rA[index], tA[index], k_int)
  multitrace_ops.append(GeneralizedBlockedOperator([[mfie, eta * efie],[-1/eta * efie, mfie]]))

  efie0 = bempp.api.operators.boundary.maxwell.electric_field(dA[index], rA[index], tA[index], k_ext)
  mfie0 = bempp.api.operators.boundary.maxwell.magnetic_field(dA[index], rA[index], tA[index], k_ext)
  multitrace_ops0.append(GeneralizedBlockedOperator([[mfie0, efie0],[-efie0, mfie0]]))
  

# Define the final operator

block_system = [(M-1) * [None] for _ in range(M-1)]

for i in range(M-1):
  for j in range(M-1):
    if i == j:
      block_system[i][j] = (multitrace_ops[i] + multitrace_ops0[i])
    else:
      efie0 = bempp.api.operators.boundary.maxwell.electric_field(dA[j], rA[i], tA[i], k_ext)
      mfie0 = bempp.api.operators.boundary.maxwell.magnetic_field(dA[j], rA[i], tA[i], k_ext)
      block_system[i][j]= GeneralizedBlockedOperator([[mfie0, efie0],[-efie0, mfie0]])
            
lhs_op = GeneralizedBlockedOperator(block_system)
op_wf = lhs_op.weak_form()

rhs = []
for i in range(M-1):
    rhs += [bempp.api.GridFunction(rA[i], dual_space = tA[i], fun=tangential_trace),
      bempp.api.GridFunction(rA[i], dual_space = tA[i], fun=neumann_trace)]

b = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(rhs, lhs_op.dual_to_range_spaces)
N = b.shape[0]
A = bempp.api.as_matrix(op_wf)
x = np.linalg.solve(A,b)

sol = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(x.ravel(), lhs_op.domain_spaces)

/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: CompilerWarning:

Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.

/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:499: CompilerWarning:

Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.



LogicError: create_buffer failed: INVALID_BUFFER_SIZE